## Stock Price Prediction

In [ ]:
# Import requirements
import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

try:
    import yfinance as yf
except ImportError:
    !pip install yfinance
    import yfinance as yf

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [ ]:
# Download dataset
today = datetime.date.today()

d1 = today.strftime('%Y-%m-%d')
end_date = d1

d2 = (today - datetime.timedelta(days=768)).strftime('%Y-%m-%d')
start_date = d2

df = yf.download('AAPL',
                start=start_date,
                end=end_date,
                progress=False)
df.reset_index(inplace=True)

In [ ]:
# Show data tail
df.tail()

In [ ]:
# Show shape
df.shape

In [ ]:
# Plot graf
figure = go.Figure(data=[go.Candlestick(x=df['Date'],
                                       open=df['Open'],
                                       high=df['High'],
                                       low=df['Low'],
                                       close=df['Close'])])
figure.update_layout(title='Apple Price Analizy',
                    xaxis_rangeslider_visible=False)
figure.show()

In [ ]:
# Find correlation
corr = df.corr()
corr['Close'].sort_values(ascending=False)

In [ ]:
# Prepare dataset
X = df.drop(['Date', 'Close'], axis=1).values
y = df['Close'].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=28)

In [ ]:
# Create the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

In [ ]:
# Summary
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=1)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
# Plot the predictions on the test data
test_predict = model.predict(X_test)

plt.plot(test_predict, label='Predictions')
plt.plot(y_test, label='Actual')
plt.legend()
plt.show()